<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C2-Basic_Networking_Tools/TCP_Proxy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *__Building a TCP Proxy__*

There are several reasons to have a __TCP__ proxy in your tool belt. You might use one for forwarding traffic to bounce from host to host, or when assessing network based software. When performing penetration tests in enterprise environments, you probably won't be able to run __Wireshark__; nor will you be able to load drivers to sniff the loopback on Windows, and network segmentation will prevent you from running your tools directly against your target host. We've built simple Python proxies, like this one, in various cases to help you understand unknown protocols, modify traffic being sent to an application, and create test cases for fuzzers. 

The proxy has a few moving parts. Let's summarize the four main functions we need to write. We need to display the communication between the local and remote machines to the console (__hexdump__). We need to receive data from an incoming socket from either the local or remote machine (__receive_from__). We need to manage the traffic direction between remote and local machines (__proxy_handler__). Finally, we need to set up a listening socket and pass it to our __proxy_handler__ (__server_loop__).

Let's get to it. Open a new file called __proxy.py__:

In [ ]:
import sys
import socket
import threading

HEX_FILTER = ''.join(
    [len(repr(chr(i))) == 3) and chr(i) or '.' for i in range(256)]) #1

def hexdump(src, length=16, show=True):
    if isinstance(src, bytes): #2
        src = src.decode()

    results = list()
    for i in range(0, len(src), length):
        word = str(src[i:i+length]) #3

        printable = word.translate(HEX_FILTER) #4
        hexa = ' '.join([f"{ord(c):02X}" for c in word])
        hexwidth = length * 3
        results.append(f"{i:04x}  {hexa:<{hexwidth}}  {printable}") #5
    if show:
        for line in results:
            print(line)
    else:
        return results

We start with a few imports. Then we define a __hexdump__ function that takes some input as bytes or a string and prints a __hexdump__ to the console. That is, it will output the packet details with both their hexadecimal values and __ASCII__ printable characters. This is useful for understanding unknown protocols, finding user credentials in plaintext protocols, and much more. We create a __HEX_FILTER__ string __[1]__ that contains __ASCII__ printable characters, if one exists, or a dot (.) if such a representation doesn't exist. For an example of what this string could contain, let's look at the character representations of two integers, __30__ and __65__, in an interactive Python shell:
```
>>> chr(65)
'A'
>>> chr(30)
'\x1e'
>>> len(repr(chr(65)))
3
>>> len(repr(chr(30)))
6
```

The character representation of __65__ is printable and the character representation of __30__ is not. As you can see, the representation of the printable character ha s a length of __3__. We use that fact to create the final __HEX_FILTER__ string: Provide the character if possible and a dot (.) if not.

The list comprehension used to create the string employs a __Boolean short circuit__ technique, which sounds pretty fancy. Let's break it down: For each integer in the range of __0__ to __255__, if the length of the corresponding character equals __3__, we get the character (chr(i)). Otherwise, we get a dot (.). Then we __join__ that list into a string so it looks something like this:
```
'............................... !"#&\'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[.]^_`abcdefghijklmnopqrstuvwxyz{|}~.................................. OTHER WEIRD CHARACTERS FOR YOU MAH NIGGAS'
```

The list comprehension gives a printable character representation of the first 256 integers. Now we can create the __hexdump__ function. First, we make sure we have a string, decoding the bytes if a byte string was passed in __[2]__. Then we grab a piece of the string to dump and put it into the __word__ variable __[3]__. We use the __translate__ built-in function to substitute the string representation of each character for the corresponding character in the raw string (__printable__) __[4]__. Likewise, we substitute the hex representation of the integer value of every chacater in the raw string (__hexa__). Finally, we create a new array to hold the strings, __result__, that contains the hex value of the index of the first byte in the word, the hex value of the word, and its printable representation __[5]__. The ouput looks like this:
```
>>> hexdump('python rocks\n and proxies roll\n')
0000 70 79 74 68 6F 6E 20 72 6F 63 6B 73 0A 20 61 6E python rocks. an
0010 64 20 70 72 6F 78 69 65 73 20 72 6F 6C 6C 0A    d proxies roll.
```

This function provides us with a way to watch the communication going throught the proxy in real time. Now let's create a function that the two ends of the proxy will use to receive data:

In [ ]:
def receive_from(connection):
    buffer = b''
    connection.settimeout(5)
    try:
        while True:
            data = connection.recv(4096)
            if not data:
                break
            buffer += data
    except Exception as e:
        pass
    return buffer

For receiving both local and remote data, we pass in the socket object to be used. We create an empty byte string, __buffer__, that will accumulate responses from the socket __[1]__. By default, we set a five second time-out, which might be aggressive if you're proxying traffic to other countries or over lossy networks, so increase the time-out as necessary. We set up a loop to read response data into the __buffer__ __[2]__ until there's no more data or we time out. Finally, we return the __buffer__ byte string to the caller, which could be either the local or remote machine.

Sometimes you may want to modify the response or request packets before the proxy sends them on their way. Let's add a couple of functions (__request_handler__ and __response_handler__) to do just that:

In [ ]:
def request_handler(buffer):
    # Perform packet modifications
    return buffer

def response_handler(buffer):
    # Perform packet modifications
    return buffer

Inside these functions, you can modify the packet contents, perform fuzzing tasks, test for authentication issues, or do whatever else your heart desires. This can be useful, for example, if you find plain-text user credentials being sent and want to try to elevate privileges on an application by passing in __admin__ instead of your own username.

Let's dive into the __proxy_handler__ function now by adding this code:

In [ ]:
def proxy_handler(client_socket, remote_host, remote_port, receive_first):
    remote_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    remote_socket.connect((remote_host, remote_port)) #1

    if receive_first: #2
        remote_buffer = receive_from(remote_socket)
        hexdump(remove_buffer)

    remote_buffer = response_handler(remote_buffer) #3
    if len(remote_buffer):
        print("[<==] Sending %d bytes to localhost." % len(remote_buffer))
        client_socket.send(remote_buffer)

    while True:
        local_buffer = receive_from(client_socket)
        if len(local_buffer):
            line = "[==>] Received %d bytes from localhost." % len(local_buffer)
            print(line)
            hexdump(local_buffer)

            local_buffer = request_handler(local_buffer)
            remote_socket.send(local_buffer)
            print("[==>] Sent to remote.")

        remote_buffer = receive_from(remote_socket)
        if len(remote_buffer):
            print("[<==] Received %d byte from remote." % len(remote_buffer))
            hexdump(remote_buffer)

            remote_buffer = response_handler(remote_buffer)
            client_socket.send(remote_buffer)
            print("[<==] Sent to localhost.")

        if not len(local_buffer) or not len(remote_buffer): #4
            client_socket.close()
            remote_socket.close()
            print("[*] No more data. Closing connections.")
            break

This function contains the bulk of the logic for our proxy. To start off, we connect to the remote host __[1]__. Then we check to make sure we don't need to first initiate a connection to the remote side and request data before going inot the main loop __[2]__. Some server daemons will expect you yo do this (__FTP__ servers typically send a banner first, for example). We then use the __receive_from__ function for both sides of the communication. It accepts a connected socket object and performs a receive. We dump the contents of the packet so that we can inspect it for anything interesting. Next, we hand the output to the __response_handler__ function __[3]__ and then send the received buffer to the local client. The rest of the proxy code is straightforward: We set up our loop to continually read from the local client, process the data, send it to the remote client, read from the remote client, process the data, and send it to the local client until we no longer detect any data. When there's no more data to send on either side of the connection __[4]__, we close both the local and remote sockets and break out of the loop.

Let's put together the __server_loop__ function to set up and manage the connection:

In [ ]:
def server_loop(local_host, local_port, remote_host, remote_port, receive_first):
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM) #1
    try:
        server.bind((local_host, local_port)) #2
    except Exception as e:
        print("Problem on bind: %r" % e)

        print("[!!] Failed to listen on %s:%d" % (local_host, local_port))
        print("[!!] Check for other listening sockets or correct permissions.")
        sys.exit(0)

    print("[*] Listening on %s:%d" % (local_host, local_port))
    server.listen(5)
    while True: #3
        client_socket, addr = server.accept()
        # Print out the local connection information
        line = "> Received incoming connection from %s:%d" % (addr[0], addr[1])
        print(line)
        # Start a thread to talk to the remote host
        proxy_thread = threading.Thread( #4
            target=proxy_handler,
            args=(client_socket, remote_host, remote_port, receive_first)
        )
        proxy_thread.start()

The __server_loop__ function creates a socket __[1]__ and then binds to the local host and listens __[2]__. In the main loop __[3]__, when a fresh connection request comes in, we hand it off to the __proxy_handler__ in a new thread __[4]__, which does all of the sending and receiving of juicy bits to either side of the data stream.

The only part left to write is the __main__ function:

In [ ]:
def main():
    if len(sys.args[1:]) != 5:
        print("Usage: ./proxy.py [localhost] [localport]", end='')
        print("[remotehost] [remoteport] [receive_first]")
        print("Example: ./proxy.py 127.0.0.1 9000 10.12.132.1 9000 True")
        sys.exit(0)

    local_host = sys.argv[1]
    local_port = int(sus.argv[2])
    
    remote_host = sys.argv[3]
    remote_port = int(sys.argv[4])

    receive_first = sys.argv[5]

    if "True" in receive_first:
        receive_first = True
    else:
        receive_first = False

    server_loop(local_host, local_port, remote_host, remote_port, receive_first)

if __name__ == "__main__":
    main()

In the __main__ function, we take in some command line arguments and then fire up the server loop that listens for connections.

# *__Kicking the Tires__*

Now that we have the core proxy loop and the supporting functions in place, let's test it against an __FTP__ server. Fire up the proxy with the following options:
```
tim@kali: sudo python proxy.py 192.168.1.203 21 ftp.sun.ac.za 21 True
```

We used __sudo__ here because port 21 is a privileged port, so listening on it requires administrative or root privileges. Now launch any __FTP__ client and set it to use localhost and port 21 as its remote host and port. Of course, you'll want to point your proxy to an __FTP__ server that will actually respond yo you. When we ran this against a test __FTP__ server, we got the following result:
```
[*] Listening on 192.168.1.203:21
> Received incoming connection from 192.168.1.203:47360
[<==] Received 30 bytes from remote.
0000  32 32 30 20 57 65 6C 63 6F 6D 65 20 74 6F 20 66   220 Welcome to f
0010  74 70 2E 73 75 6E 2E 61 63 2E 7A 61 0D 0A         tp.sun.ac.za..
0000  55 53 45 52 20 61 6E 6F 6E 79 6D 6F 75 73 0D 0A   USER anonymous..
0000  33 33 31 20 50 6C 65 61 73 65 20 73 70 65 63 69   331 Please speci
0010  66 79 20 74 68 65 20 70 61 73 73 77 6F 72 64 2E   fy the password.
0020  0D 0A                                             ..
0000  50 41 53 53 20 73 65 6B 72 65 74 0D 0A            PASS sekret..
0000  32 33 30 20 4C 6F 67 69 6E 20 73 75 63 63 65 73   230 Login succes
0010  73 66 75 6C 2E 0D 0A                              sful...
[==>] Sent to local.
[<==] Received 6 bytes from local.
0000  53 59 53 54 0D 0A                                 SYST..
0000  32 31 35 20 55 4E 49 58 20 54 79 70 65 3A 20 4C   215 UNIX Type: L
0010  38 0D 0A                                          8..
[<==] Received 28 bytes from local.
0000  50 4F 52 54 20 31 39 32 2C 31 36 38 2C 31 2C 32   PORT 192,168,1,2
0010  30 33 2C 31 38 37 2C 32 32 33 0D 0A               03, 187,223..
0000  32 30 30 20 50 4F 52 54 20 63 6F 6D 6D 61 6E 64   200 PORT command
0010  20 73 75 63 63 65 73 73 66 75 6C 2E 20 43 6F 6E    successful. Con
0020  73 69 64 65 72 20 75 73 69 6E 67 20 50 41 53 56   sider using PASV
0030  2E 0D 0A                                          ...
[<==] Received 6 bytes from local.
0000  4C 49 53 54 0D 0A                                 150 Here comes t
0010  68 65 20 64 69 72 65 63 74 6F 72 79 20 6C 69 73   he directory lis
0020  74 69 6E 67 2E 0D 0A 32 32 36 20 44 69 72 65 63   ting...226 Direc
0030  74 6F 72 79 20 73 65 6E 64 20 4F 4B 2E 0D 0A      tory send OK...
0000  50 4F 52 54 20 31 39 32 2C 31 36 38 2C 31 2C 32   PORT 192,168,1,2
0010  30 33 2C 32 31 38 2C 31 31 0D 0A                  03,218,11..
0000  32 30 30 20 50 4F 52 54 20 63 6F 6D 6D 61 6E 64   200 PORT command
0010  20 73 75 63 63 65 73 73 66 75 6C 2E 20 43 6F 6E    successful. Con
0020  73 69 64 65 72 20 75 73 69 6E 67 20 50 41 53 56   sider using PASV
0030  2E 0D 0A                                          ...
0000  51 55 49 54 0D 0A                                 QUIT..
[==>] Sent to remote.
0000  32 32 31 20 47 6F 6F 64 62 79 65 2E 0D 0A         221 Goodbye...
[==>] Sent to local.
[*] No more data. Closing connections.
```

In another terminal on the Kali machine, we started an __FTP__ session to the Kali machine's IP address using the default port, 21:
```
tim@kali:$ ftp 192.168.1.203
Connected to 192.168.1.203.
220 Welcome to ftp.sun.ac.za
Name (192.168.1.203:tim): anonymous
331 Please specify the password.
Password:
230 Login successful.
Remote system type is UNIX.
Using binary mode to transfer files.
ftp> ls
200 PORT command successful. Consider using PASV.
150 Here comes the directory listing.
lrwxrwxrwx    1 1001     1001          48 Jul 17  2008 CPAN -> pub/mirrors/ftp.funet.fi/pub/languages/perl/CPAN
lrwxrwxrwx    1 1001     1001          21 Oct 21  2009 CRAN -> pub/mirrors/ubuntu.com
drwxr-xr-x    2 1001     1001        4096 Apr 03  2019 veeam
drwxr-xr-x    6 1001     1001        4096 Jun 27  2016 win32InetKeyTeraTerm226 Directory send OK.
ftp> bye
221 Goodbye.
```

You can clearly see that we're able to successfully receive the __FTP__ banner and send in a username and password, and that it cleanly exits.